<a href="https://colab.research.google.com/github/ShwetaRoy17/GC-LFF/blob/main/hare_krishna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
!pip install tensorflow numpy

In [ ]:
import random, time, psutil, threading, csv
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import optimizers, layers

from keras.api.layers import Dense
from keras.api.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import entropy, ttest_rel, wilcoxon

# Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
aqi_df = pd.read_csv('/content/drive/MyDrive/city/city_hour.csv')

In [ ]:
# content
# aqi_df = pd.read_csv('/content/city_hour.csv')

In [ ]:
# Label Encoding for "City"
label_encoder = LabelEncoder()
aqi_df['city'] = label_encoder.fit_transform(aqi_df['City'])

In [ ]:
# Create a mapping dictionary
city_mapping = dict(zip( label_encoder.transform(label_encoder.classes_),label_encoder.classes_))

# Output the mapping
print("City Mapping:", city_mapping)

In [ ]:
aqi_df.drop(columns=['City'], inplace=True)
aqi_df.drop(columns=['AQI_Bucket'], inplace=True)

In [ ]:
# Calculate missing values
missing_values_count = aqi_df.isnull().sum()
total_rows = len(aqi_df)
# Create a DataFrame to display missing values information
missing_data_table = pd.DataFrame({
    "Feature Name": missing_values_count.index,
    "Missing Values Count": missing_values_count.values,
    "Percentage of Missing Data": (missing_values_count.values / total_rows) * 100
}).sort_values(by="Percentage of Missing Data", ascending=False)


missing_data_table
# Save the table to a CSV file for further analysis
# missing_data_table.to_csv('missing_data_summary.csv', index=False)

# Data prepocessing

In [ ]:
# Feature scaling
# scaler = MinMaxScaler()
# data_scaled = pd.DataFrame(scaler.fit_transform(data.drop(columns=['Date', 'city'])),
#                            columns=data.columns.drop(['Date', 'city']))
# data_scaled['city'] = data['city']
# data_scaled['Date'] = data['Date']

In [ ]:
aqi_df['date'] = pd.to_datetime(aqi_df['Datetime'])
aqi_df['month'] = aqi_df['date'].dt.month
aqi_df['year'] = aqi_df['date'].dt.year
aqi_df['day_of_week'] = aqi_df['date'].dt.dayofweek

In [ ]:
aqi_df.drop(columns=['Datetime'], inplace=True)

In [ ]:
# Split city-wise data
cities = aqi_df['city'].unique()
print(cities)

In [ ]:
# city_data = {city: data_scaled[data_scaled['city'] == city].drop(columns=['city', 'date']) for city in cities}

In [ ]:
# print(city_data[0])

In [ ]:
aqi_df.columns

# Missing Data

In [ ]:
cols = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2',
        'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI']

# For final training, use the full dataset (which already contains missing values)
data_incomplete = aqi_df[cols].copy()

# For hyperparameter tuning, extract complete rows only
data_complete = aqi_df[cols].dropna().copy()
print("Total complete cases for tuning:", data_complete.shape[0])

# Function to artificially introduce missingness on a complete dataset.
def introduce_missingness(data, missing_rate=0.2, random_state=42):
    np.random.seed(random_state)
    data_missing = data.copy().values
    mask = np.random.rand(*data_missing.shape) < missing_rate  # True indicates missing
    data_missing[mask] = np.nan
    return data_missing, mask

# Create artificially masked data from the complete subset
data_complete_missing, mask_complete = introduce_missingness(data_complete, missing_rate=0.2, random_state=42)

# Splitting Data for hypertuning

In [ ]:
train_data, val_data, train_missing, val_missing, train_mask, val_mask = train_test_split(
    data_complete.values, data_complete_missing, mask_complete, test_size=0.2, random_state=42)

# Evaluation Metric Function

In [ ]:
def compute_metrics(true, imputed, missing_mask):
    # Evaluate only on positions that were artificially masked (where missing_mask is True)
    mse = mean_squared_error(true[missing_mask], imputed[missing_mask])
    rmse = np.sqrt(mse)

    # Compute KL divergence per feature using histogram estimates
    kl_divs = []
    for col in range(true.shape[1]):
        true_vals = true[missing_mask[:, col], col]
        imputed_vals = imputed[missing_mask[:, col], col]
        if true_vals.size == 0 or imputed_vals.size == 0:
            continue
        true_hist, _ = np.histogram(true_vals, bins=50, density=True)
        imputed_hist, _ = np.histogram(imputed_vals, bins=50, density=True)
        true_hist += 1e-8
        imputed_hist += 1e-8
        kl = entropy(true_hist, imputed_hist)
        kl_divs.append(kl)
    kl_div = np.mean(kl_divs) if kl_divs else np.nan
    return mse, rmse, kl_div

# GAIN

In [ ]:
class GAIN:
    def __init__(self, data_miss, hint_rate=0.9, alpha=100, batch_size=256, epochs=1000, learning_rate=1e-3):
        """
        data_miss: numpy array with missing values as np.nan
        hint_rate: probability of providing hint information
        alpha: weight for the reconstruction loss
        batch_size: training batch size
        epochs: number of training epochs
        learning_rate: learning rate for optimizers
        """
        self.data_miss = data_miss
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.n_samples, self.dim = data_miss.shape

        # Normalize data feature-wise to [0, 1]
        self.min_val = np.nanmin(data_miss, axis=0)
        self.max_val = np.nanmax(data_miss, axis=0)
        self.norm_data = (data_miss - self.min_val) / (self.max_val - self.min_val + 1e-8)
        self.mask = (~np.isnan(data_miss)).astype(np.float32)
        self.norm_data = np.nan_to_num(self.norm_data, 0)

    def build_generator(self):
        inputs = layers.Input(shape=(self.dim*2,))
        h = layers.Dense(128, activation='relu')(inputs)
        h = layers.Dense(128, activation='relu')(h)
        out = layers.Dense(self.dim, activation='sigmoid')(h)
        model = tf.keras.models.Model(inputs, out)
        return model

    def build_discriminator(self):
        inputs = layers.Input(shape=(self.dim*2,))
        h = layers.Dense(128, activation='relu')(inputs)
        h = layers.Dense(128, activation='relu')(h)
        out = layers.Dense(self.dim, activation='sigmoid')(h)
        model = tf.keras.models.Model(inputs, out)
        return model

    def train(self, verbose=False):
        X = self.norm_data.astype(np.float32)
        M = self.mask.astype(np.float32)

        # Build generator and discriminator models
        generator = self.build_generator()
        discriminator = self.build_discriminator()

        G_optimizer = optimizers.Adam(self.learning_rate)
        D_optimizer = optimizers.Adam(self.learning_rate)

        # Training loop
        for epoch in range(self.epochs):
            # Sample a batch with replacement
            idx = np.random.choice(self.n_samples, self.batch_size, replace=True)
            X_batch = X[idx]
            M_batch = M[idx]

            # Add small random noise for missing entries
            Z = np.random.uniform(0, 0.01, size=[self.batch_size, self.dim]).astype(np.float32)
            X_batch_noisy = X_batch * M_batch + Z * (1 - M_batch)

            # Create a hint matrix
            H_batch = (np.random.uniform(0, 1, size=[self.batch_size, self.dim]) < self.hint_rate).astype(np.float32)
            H_batch = M_batch * H_batch

            # --- Train Generator ---
            with tf.GradientTape() as tape:
                G_input = tf.concat([X_batch_noisy, M_batch], axis=1)
                G_sample = generator(G_input)
                X_hat = X_batch * M_batch + G_sample * (1 - M_batch)
                D_input_for_G = tf.concat([X_hat, H_batch], axis=1)
                D_prob = discriminator(D_input_for_G)
                # Loss: adversarial loss on missing entries plus weighted reconstruction loss on observed entries
                G_loss_adv = -tf.reduce_mean((1 - M_batch) * tf.math.log(D_prob + 1e-8))
                mse_loss = tf.reduce_mean((M_batch * X_batch - M_batch * G_sample)**2) / (tf.reduce_mean(M_batch) + 1e-8)
                G_loss = G_loss_adv + self.alpha * mse_loss

            G_gradients = tape.gradient(G_loss, generator.trainable_variables)
            G_optimizer.apply_gradients(zip(G_gradients, generator.trainable_variables))

            # --- Train Discriminator ---
            with tf.GradientTape() as tape:
                G_input = tf.concat([X_batch_noisy, M_batch], axis=1)
                G_sample = generator(G_input)
                X_hat = X_batch * M_batch + G_sample * (1 - M_batch)
                D_input = tf.concat([X_hat, H_batch], axis=1)
                D_prob = discriminator(D_input)
                D_loss = -tf.reduce_mean(M_batch * tf.math.log(D_prob + 1e-8) + (1 - M_batch) * tf.math.log(1 - D_prob + 1e-8))

            D_gradients = tape.gradient(D_loss, discriminator.trainable_variables)
            D_optimizer.apply_gradients(zip(D_gradients, discriminator.trainable_variables))

            if verbose and epoch % 100 == 0:
                print(f"Epoch {epoch} | G_loss: {G_loss.numpy():.4f} | D_loss: {D_loss.numpy():.4f}")

        # After training, impute missing values for the whole dataset
        X_tensor = tf.convert_to_tensor(X)
        M_tensor = tf.convert_to_tensor(M)
        G_input_full = tf.concat([X_tensor, M_tensor], axis=1)
        G_sample_full = generator(G_input_full)
        X_imputed_norm = X * M + G_sample_full.numpy() * (1 - M)
        X_imputed = X_imputed_norm * (self.max_val - self.min_val + 1e-8) + self.min_val
        return X_imputed

# Hyperparameter tuning

In [ ]:
# Hyperparameter grid
alpha_list = [50, 100, 150, 200, 250]
hint_rate_list = [0.8, 0.9, 0.95]
learning_rate_list = [1e-3, 5e-4, 1e-4]
batch_size_list = [128, 256]
tune_epochs = 500  # fewer epochs for tuning

results = []
best_rmse = np.inf
# best_params = {}
best_params = {'alpha': 200, 'hint_rate': 0.8, 'learning_rate': 0.0005, 'batch_size': 256, 'epochs': 500}

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Loop over hyperparameters
# for alpha in alpha_list:
#     for hint_rate in hint_rate_list:
#         for lr in learning_rate_list:
#             for batch_size in batch_size_list:
#                 print(f"Training with alpha={alpha}, hint_rate={hint_rate}, lr={lr}, batch_size={batch_size}")
#                 # Train on training split
#                 gain_model_train = GAIN(train_missing, hint_rate=hint_rate, alpha=alpha,
#                                           batch_size=batch_size, epochs=tune_epochs, learning_rate=lr)
#                 train_imputed = gain_model_train.train(verbose=False)
#                 # Evaluate on validation split
#                 gain_model_val = GAIN(val_missing, hint_rate=hint_rate, alpha=alpha,
#                                         batch_size=batch_size, epochs=tune_epochs, learning_rate=lr)
#                 val_imputed = gain_model_val.train(verbose=False)

#                 # Compute RMSE on the validation missing entries
#                 mse_val, rmse_val, kl_val = compute_metrics(val_data, val_imputed, val_mask)
#                 print(f"--> Validation RMSE: {rmse_val:.4f}")
#                 results.append({
#                     'alpha': alpha,
#                     'hint_rate': hint_rate,
#                     'learning_rate': lr,
#                     'batch_size': batch_size,
#                     'rmse': rmse_val,
#                     'mse': mse_val,
#                     'kl': kl_val
#                 })
#                 if rmse_val < best_rmse:
#                     best_rmse = rmse_val
#                     best_params = {
#                         'alpha': alpha,
#                         'hint_rate': hint_rate,
#                         'learning_rate': lr,
#                         'batch_size': batch_size,
#                         'epochs': tune_epochs
#                     }
print("Best hyperparameters:", best_params)
print("Best Validation RMSE:", best_rmse)

# Final Training on full dataset

In [ ]:
aqi_df.columns

In [ ]:
# Increase epochs for final training (if desired)
final_epochs = 2000
gain_final = GAIN(data_incomplete.values, hint_rate=best_params['hint_rate'],
                  alpha=best_params['alpha'], batch_size=best_params['batch_size'],
                  epochs=final_epochs, learning_rate=best_params['learning_rate'])
data_imputed_final = gain_final.train(verbose=True)
data_imputed_final_df = pd.DataFrame(data_imputed_final, columns=cols)

In [ ]:
complete_aqi_df = aqi_df.copy()
complete_aqi_df[cols] = complete_aqi_df[cols].fillna(data_imputed_final_df)

# Plot

In [ ]:
# col_to_plot = random.choice(cols)
# col_index = cols.index(col_to_plot)

# Identify missing indices using the artificial mask
# missing_indices = np.where(mask_complete[:, col_index])[0]

# plt.figure(figsize=(12, 6))
# plt.plot(data_complete.index, data_complete[col_to_plot].values, label='Original', alpha=0.7)
# plt.plot(data_complete.index[missing_indices],
#          data_imputed_final_df.loc[missing_indices, col_to_plot],
#          color='blue', label='GAN Imputed', linewidth=2, zorder=5)
# plt.xlabel("Index")
# plt.ylabel(col_to_plot)
# plt.title(f"Original vs GAN Imputed Values for {col_to_plot}")
# plt.legend()
# plt.grid(True, linestyle='--', alpha=0.6)
# plt.show()

# Plot of Missing Data


In [ ]:
# List of features to compare
features = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx']

# Create subplots for each feature
plt.figure(figsize=(15, 10))

for i, feature in enumerate(features):
    plt.subplot(3, 2, i+1)

    # Plot the original complete data for the feature as a line plot
    plt.plot(data_complete.index, data_complete[feature].values, label='Original',
             color='blue', alpha=0.7)

    # Identify the indices where artificial missingness was introduced for this feature
    feature_idx = cols.index(feature)
    missing_idx = np.where(mask_complete[:, feature_idx])[0]

    # Plot the imputed values from the final imputed DataFrame as a line plot
    plt.plot(data_complete.index[missing_idx],
             data_imputed_final_df.loc[data_complete.index[missing_idx], feature],
             label='GAIN Imputed', color='red', alpha=0.7)

    plt.title(f"{feature}: Original vs GAIN Imputed")
    plt.xlabel("Index")
    plt.ylabel(feature)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Comparison with Tradtional Methods

In [ ]:
# Mean Imputation
mean_imputer = SimpleImputer(strategy='mean')
data_mean = pd.DataFrame(mean_imputer.fit_transform(data_complete_missing), columns=cols)

# KNN Imputation
knn_imputer = KNNImputer(n_neighbors=5)
data_knn = pd.DataFrame(knn_imputer.fit_transform(data_complete_missing), columns=cols)

# MICE Imputation (IterativeImputer)
mice_imputer = IterativeImputer(random_state=0)
data_mice = pd.DataFrame(mice_imputer.fit_transform(data_complete_missing), columns=cols)

In [ ]:
# Increase epochs for final training (if desired)
final_epochs = 2000
gain_final = GAIN(data_complete_missing, hint_rate=best_params['hint_rate'],
                  alpha=best_params['alpha'], batch_size=best_params['batch_size'],
                  epochs=final_epochs, learning_rate=best_params['learning_rate'])
data_gain_imputed = gain_final.train(verbose=True)
data_gain_complete = pd.DataFrame(data_gain_imputed, columns=cols)

In [ ]:
data_substituted = aqi_df.fillna(data_gain_complete)
print(data_substituted.shape)
print(aqi_df.shape)

In [ ]:
def compute_metrics(true, imputed, missing_mask):
    # valid_mask = np.logical_and(missing_mask, ~np.isnan(true))
    valid_mask = missing_mask
    # Calculate errors only on missing entries (where missing_mask is True)
    mse = mean_squared_error(true[valid_mask], imputed[valid_mask])
    rmse = np.sqrt(mse)
    # Compute KL divergence per feature (using histogram estimates)
    kl_divs = []
    for col in range(true.shape[1]):
        true_hist, _ = np.histogram(true[valid_mask[:, col], col], bins=50, density=True)
        imputed_hist, _ = np.histogram(imputed[valid_mask[:, col], col], bins=50, density=True)
        true_hist += 1e-8
        imputed_hist += 1e-8
        kl = entropy(true_hist, imputed_hist)
        kl_divs.append(kl)
    kl_div = np.mean(kl_divs)
    return mse, rmse, kl_div

# Ground truth (complete data) as numpy array
true_data = data_complete.values

# Compute metrics for each imputation method
mse_mean, rmse_mean, kl_mean = compute_metrics(true_data, data_mean.values, mask_complete)
mse_knn, rmse_knn, kl_knn = compute_metrics(true_data, data_knn.values, mask_complete)
mse_mice, rmse_mice, kl_mice = compute_metrics(true_data, data_mice.values, mask_complete)
mse_gain, rmse_gain, kl_gain = compute_metrics(true_data, data_gain_imputed, mask_complete)

print("Mean Imputation  - MSE: {:.4f}, RMSE: {:.4f}, KL Divergence: {:.4f}".format(mse_mean, rmse_mean, kl_mean))
print("KNN Imputation   - MSE: {:.4f}, RMSE: {:.4f}, KL Divergence: {:.4f}".format(mse_knn, rmse_knn, kl_knn))
print("MICE Imputation  - MSE: {:.4f}, RMSE: {:.4f}, KL Divergence: {:.4f}".format(mse_mice, rmse_mice, kl_mice))
print("GAIN Imputation  - MSE: {:.4f}, RMSE: {:.4f}, KL Divergence: {:.4f}".format(mse_gain, rmse_gain, kl_gain))

In [ ]:
def get_error_vector(true, imputed, missing_mask):
    # Compute squared errors per entry and then average per sample (only for missing entries)
    errors = (true - imputed)**2
    # For each sample, take the mean error over features where the value was missing.
    sample_errors = np.array([np.nanmean(errors[i][missing_mask[i]]) for i in range(true.shape[0])])
    return sample_errors

error_mean = get_error_vector(true_data, data_mean.values, mask_complete)
error_knn  = get_error_vector(true_data, data_knn.values, mask_complete)
error_mice = get_error_vector(true_data, data_mice.values, mask_complete)
error_gain = get_error_vector(true_data, data_gain_imputed.values, mask_complete)

# Remove samples where there were no missing entries (if any)
valid_idx = ~np.isnan(error_gain)
error_mean = error_mean[valid_idx]
error_knn  = error_knn[valid_idx]
error_mice = error_mice[valid_idx]
error_gain = error_gain[valid_idx]

# Paired t-tests: comparing GAIN with each method
ttest_mean = ttest_rel(error_gain, error_mean)
ttest_knn  = ttest_rel(error_gain, error_knn)
ttest_mice = ttest_rel(error_gain, error_mice)

print("T-test (GAIN vs Mean)  | Statistic: {:.4f}, p-value: {:.4f}".format(ttest_mean.statistic, ttest_mean.pvalue))
print("T-test (GAIN vs KNN)   | Statistic: {:.4f}, p-value: {:.4f}".format(ttest_knn.statistic, ttest_knn.pvalue))
print("T-test (GAIN vs MICE)  | Statistic: {:.4f}, p-value: {:.4f}".format(ttest_mice.statistic, ttest_mice.pvalue))

# Wilcoxon signed-rank tests
wilcox_mean = wilcoxon(error_gain, error_mean)
wilcox_knn  = wilcoxon(error_gain, error_knn)
wilcox_mice = wilcoxon(error_gain, error_mice)

print("Wilcoxon (GAIN vs Mean)  | Statistic: {:.4f}, p-value: {:.4f}".format(wilcox_mean.statistic, wilcox_mean.pvalue))
print("Wilcoxon (GAIN vs KNN)   | Statistic: {:.4f}, p-value: {:.4f}".format(wilcox_knn.statistic, wilcox_knn.pvalue))
print("Wilcoxon (GAIN vs MICE)  | Statistic: {:.4f}, p-value: {:.4f}".format(wilcox_mice.statistic, wilcox_mice.pvalue))

#  Federated Learning Pre-req

In [ ]:
#CNN-LSTM model
# --- Step 3: Prepare Data for Federated Learning ---
def create_model(input_shape):
    model = tf.keras.Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=input_shape),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.LSTM(64, activation='relu', return_sequences=True),
        layers.LSTM(32, activation='tanh'),
        layers.Dense(10, activation='relu'),
        layers.Dense(1)  # Predict AQI
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
    return model

In [ ]:
# --- Helper function to create time-series windows ---
def create_time_series(df, window_size=10):
    # Exclude the 'AQI' column from the features
    feature_cols = df.columns.drop('AQI')
    data_features = df[feature_cols].values
    # Use the AQI column as the target
    data_target = df['AQI'].values

    X, y = [], []
    for i in range(len(df) - window_size):
        X.append(data_features[i:i+window_size, :])
        y.append(data_target[i+window_size])
    return np.array(X), np.array(y)

# Preprocessing Citywise data

In [ ]:
nodes_data = {}
nodes_test_data = {}
window_size = 15

for city in complete_aqi_df['city'].unique():
    city_df = complete_aqi_df[complete_aqi_df['city'] == city][cols]
    X, y = create_time_series(city_df, window_size=window_size)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    nodes_data[city] = (X_train, y_train)
    nodes_test_data[city] = (X_test, y_test)
    print(f"City {city} - Train Samples: {len(X_train)}, Test Samples: {len(X_test)}")

# FL: Error Evaluation

In [ ]:
# --- Evaluation ---
mse_list, rmse_list, mae_list, r2_list, rmsle_list = [], [], [], [], []
city_metrics = {}

def rmsle(y_true, y_pred):
    # Adding 1 to avoid log(0) and ensure positive values
    return np.sqrt(np.mean((np.log(y_pred + 1) - np.log(y_true + 1))**2))

print("\n--- Final Evaluation ---")
for city, (X_test, y_test) in nodes_test_data.items():
    preds = global_model.predict(X_test, verbose=0).flatten()
    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    rmsle_val = rmsle(y_test, preds)

    mse_list.append(mse)
    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)
    rmsle_list.append(rmsle_val)

    city_metrics[city] = {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'RMSLE': rmsle_val}
    print(f"{city} => MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}, RMSLE: {rmsle_val:.4f}")

In [ ]:
import time

nodes_data = {}
nodes_val_data = {}
nodes_test_data = {}
window_size = 15

# Create train/val/test splits per city
for city in complete_aqi_df['city'].unique():
    city_df = complete_aqi_df[complete_aqi_df['city'] == city][cols]
    X, y = create_time_series(city_df, window_size=window_size)

    # Split: 60% train, 20% val, 20% test
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, shuffle=False)

    nodes_data[city] = (X_train, y_train)
    nodes_val_data[city] = (X_val, y_val)
    nodes_test_data[city] = (X_test, y_test)

    print(f"City {city} - Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")



# NEW FL CODE

In [ ]:
# --- Federated Learning ---
num_rounds = 10
local_epochs = 1
batch_size = 32
example_city = list(nodes_data.keys())[0]
global_input_shape = nodes_data[example_city][0].shape[1:]

global_model = create_model(global_input_shape)
global_weights = global_model.get_weights()
training_time_record = {}

max_times = []

for rnd in range(num_rounds):
    print(f"\n--- Federated Round {rnd+1} ---")
    local_weights = []
    round_training_times = {}

    for city in nodes_data:
        X_train, y_train = nodes_data[city]

        local_model = create_model(global_input_shape)
        local_model.set_weights(global_weights)

        start_time = time.time()
        local_model.fit(X_train, y_train, epochs=local_epochs, batch_size=batch_size, verbose=0)
        elapsed_time = time.time() - start_time

        round_training_times[city] = elapsed_time
        local_weights.append((len(X_train), local_model.get_weights()))

        print(f"City {city}: Samples = {len(X_train)}, Time = {elapsed_time:.2f} sec")

    training_time_record[rnd+1] = round_training_times

    round_max = max(round_training_times.values())
    max_times.append(round_max)

    # Weighted FedAvg
    total_samples = sum([n for n, _ in local_weights])
    new_global_weights = []
    for weights_per_layer in zip(*[w[1] for w in local_weights]):
        weighted_avg = np.sum([n / total_samples * w for (n, w) in zip([n for n, _ in local_weights], weights_per_layer)], axis=0)
        new_global_weights.append(weighted_avg)
    global_weights = new_global_weights

global_model.set_weights(global_weights)

In [ ]:
max_times = [14.653037548065186,
 13.506239414215088,
 9.634636878967285,
 15.834002494812012,
 17.275151014328003,
 9.323030710220337,
 8.875787496566772,
 21.47063636779785,
 8.884452104568481,
 22.44563364982605]

In [ ]:
# --- Personalized Fine-Tuning ---
print("\n--- Personalized Fine-Tuning Phase ---")
personalized_models = {}
personalized_results = {}
personalized_times = {}

for city in nodes_data:
    X_train, y_train = nodes_data[city]
    X_val, y_val = nodes_val_data[city]
    X_test, y_test = nodes_test_data[city]

    personal_model = create_model(global_input_shape)
    personal_model.set_weights(global_weights)

    # Combine train + val for fine-tuning
    X_personal = np.concatenate([X_train, X_val], axis=0)
    y_personal = np.concatenate([y_train, y_val], axis=0)

    start_time = time.time()
    personal_model.fit(X_personal, y_personal, epochs=3, batch_size=batch_size, verbose=0)
    elapsed_time = time.time() - start_time

    test_loss,test_mse, test_mae = personal_model.evaluate(X_test, y_test, verbose=0)
    print()

    personalized_models[city] = personal_model
    personalized_results[city] = {
        "loss": test_loss,
        "mae": test_mae,
        "time": elapsed_time
    }
    personalized_times[city] = elapsed_time

    print(f"City {city}: Personalized Test Loss = {test_loss:.4f}, MAE = {test_mae:.4f}, Fine-tune Time = {elapsed_time:.2f} sec")

In [ ]:
# --- Personalized Model Evaluation ---
personal_mse_list, personal_rmse_list, personal_mae_list = [], [], []
personal_r2_list, personal_rmsle_list = [], []
personalized_city_metrics = {}

def rmsle(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

print("\n--- Personalized Model Evaluation ---")
for city in nodes_test_data:
    X_test, y_test = nodes_test_data[city]
    model = personalized_models[city]
    preds = model.predict(X_test, verbose=0).flatten()

    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    rmsle_val = rmsle(y_test, preds)

    personal_mse_list.append(mse)
    personal_rmse_list.append(rmse)
    personal_mae_list.append(mae)
    personal_r2_list.append(r2)
    personal_rmsle_list.append(rmsle_val)

    personalized_city_metrics[city] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'RMSLE': rmsle_val,
        'FineTuneTime': personalized_results[city]['time']
    }

    print(f"{city} => MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}, RMSLE: {rmsle_val:.4f}, Time: {personalized_results[city]['time']:.2f} sec")


# FL: Error Plot

In [ ]:
# --- Confidence Intervals / Error Bars ---
def summary_stat(metric_list, name):
    mean_val = np.mean(metric_list)
    std_val = np.std(metric_list)
    print(f"\n{name} Mean: {mean_val:.4f}, Std: {std_val:.4f}")
    return mean_val, std_val

mean_mse, std_mse = summary_stat(personal_mse_list, "MSE")
mean_rmse, std_rmse = summary_stat(personal_rmse_list, "RMSE")
mean_mae, std_mae = summary_stat(personal_mae_list, "MAE")

# --- Visualization with Error Bars ---
plt.figure(figsize=(10, 6))
cities = list(personalized_city_metrics.keys())
mse_vals = [personalized_city_metrics[c]['MSE'] for c in cities]
rmse_vals = [personalized_city_metrics[c]['RMSE'] for c in cities]
mae_vals = [personalized_city_metrics[c]['MAE'] for c in cities]

x = np.arange(len(cities))
width = 0.25

plt.bar(x - width, mse_vals, width, label='MSE')
plt.bar(x, rmse_vals, width, label='RMSE')
plt.bar(x + width, mae_vals, width, label='MAE')
plt.xticks(x, cities, rotation=45)
plt.ylabel("Error")
plt.title("Model Performance by City")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# --- Confidence Intervals / Error Bars ---
def summary_stat(metric_list, name):
    mean_val = np.mean(metric_list)
    std_val = np.std(metric_list)
    print(f"\n{name} Mean: {mean_val:.4f}, Std: {std_val:.4f}")
    return mean_val, std_val

mean_mse, std_mse = summary_stat(mse_list, "MSE")
mean_rmse, std_rmse = summary_stat(rmse_list, "RMSE")
mean_mae, std_mae = summary_stat(mae_list, "MAE")

# --- Visualization with Error Bars ---
plt.figure(figsize=(10, 6))
cities = list(city_metrics.keys())
mse_vals = [city_metrics[c]['MSE'] for c in cities]
rmse_vals = [city_metrics[c]['RMSE'] for c in cities]
mae_vals = [city_metrics[c]['MAE'] for c in cities]

x = np.arange(len(cities))
width = 0.25

plt.bar(x - width, mse_vals, width, label='MSE')
plt.bar(x, rmse_vals, width, label='RMSE')
plt.bar(x + width, mae_vals, width, label='MAE')
plt.xticks(x, cities, rotation=45)
plt.ylabel("Error")
plt.title("Model Performance by City")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def plot_prediction_vs_actual(city_name, y_true, y_pred, num_points=100):
    """
    Plots the predicted vs actual AQI values for a given city.

    Args:
        city_name (str): Name of the city.
        y_true (np.array): Ground truth AQI values.
        y_pred (np.array): Predicted AQI values.
        num_points (int): Number of data points to show (default: 100).
    """
    # Limit the number of points plotted for readability
    y_true = y_true[:num_points]
    y_pred = y_pred[:num_points]

    plt.figure(figsize=(12, 6))
    plt.plot(y_true, label='Actual AQI', marker='o')
    plt.plot(y_pred, label='Predicted AQI', marker='x')
    plt.title(f"AQI Prediction vs Actual - {city_mapping[city_name]}")
    plt.xlabel("Time Step")
    plt.ylabel("AQI")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

cities=[2,10,12,14,18,19,20,25]

for city in range(25):
  # Get test data for the city
  X_test, y_test = nodes_test_data[city]

  # Predict using the final global model
  y_pred = global_model.predict(X_test).flatten()

  # Call the function to visualize predictions with error bars
  plot_prediction_vs_actual(city_name=city, y_true=y_test, y_pred=y_pred, num_points=100)

In [ ]:
# --- Training Time Output ---
print("\nTraining Time Summary:")
for round_num, times in training_time_record.items():
    print(f"Round {round_num}: {times}")

# Centralized Learning

In [ ]:
# --- Centralized Learning (Corrected) ---

# Create time series data from the centralized (complete) dataset
# Use the same window size as before and the preprocessed features in 'cols'
window_size = 15
X, y = create_time_series(complete_aqi_df[cols], window_size=window_size)

# Optionally, you could split X and y into train/test sets.
# Here we use the full dataset for demonstration.
print("Centralized Data Shape:", X.shape, y.shape)

# Create the model using the same architecture
model = create_model(input_shape=X.shape[1:])
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

# Train the model (adjust epochs as needed)
# model.fit(X, y, epochs=10, batch_size=32, verbose=1)

# 1) Prepare to collect times
# run_times = [1083.4012246131897, 1087.9414508342743, 1084.7454762458801, 1055.643892288208, 974.6234679222107, 987.1062573756184, 976.25, 980.28, 983.14, 973.76]

# run_times = [983.14, 1055.643892288208, 973.76, 1084.7454762458801,
#  976.25, 1087.9414508342743, 987.1062573756184, 980.28,
#  974.6234679222107, 1083.4012246131897]


# 2) Repeat fit 10×
for run in range(10 - len(run_times)):
    print(f"\n--- Centralized Run {run} ---")
    start = time.time()
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)
    elapsed = time.time() - start

    print(f"Run {run} time: {elapsed:.2f} sec")
    run_times.append(elapsed)

In [ ]:
run_times = [983.14, 1055.643892288208, 973.76, 1084.7454762458801,
 976.25, 1087.9414508342743, 987.1062573756184, 980.28,
 974.6234679222107, 1083.4012246131897]


In [ ]:
run_times

In [ ]:
# your two timing lists
fl_times = max_times        # e.g. [t1, t2, …, t10]
cl_times = run_times        # e.g. [t1', t2', …, t10']

# x‐axes (1→10)
rounds = list(range(1, len(fl_times) + 1))
runs   = list(range(1, len(cl_times) + 1))

plt.figure()
plt.plot(rounds, fl_times, marker='o', label='Federated Learning')
plt.plot(runs,   cl_times, marker='s', label='Centralized Learning')
plt.xlabel('Iteration (Round)')
plt.ylabel('Training Time (sec)')
plt.title('FL vs. CL: Training Time Comparison')
plt.legend()
plt.grid(True)
plt.show()

# CL vs FL city wise prediction graph

In [ ]:
for city in nodes_test_data:
    city_name = city_mapping[city]

    # --- FL Predictions ---
    X_test, y_test = nodes_test_data[city]
    fl_model = personalized_models[city]
    fl_preds = fl_model.predict(X_test, verbose=0).flatten()

    # --- Centralized Predictions ---
    # Filter the complete dataset for the current city.
    city_df = complete_aqi_df[complete_aqi_df['city'] == city][cols]
    # Create time-series data for the city.
    X_cent, y_cent = create_time_series(city_df, window_size=window_size)
    cent_preds = model.predict(X_cent, batch_size=32, verbose=0).flatten()

    # --- Align the Curves ---
    # It is likely that the lengths of the FL test set and the CL test set differ.
    # We take the minimum length among original values and predictions from both models.
    common_len = min(len(y_test), len(fl_preds), len(y_cent), len(cent_preds))
    y_test_common = y_test[:common_len]
    fl_preds_common = fl_preds[:common_len]
    y_cent_common = y_cent[:common_len]
    cent_preds_common = cent_preds[:common_len]

    # Optionally, if you believe y_test and y_cent represent the same underlying AQI,
    # you can choose one as the "Original" curve. Here, we use y_test_common.

    # --- Plotting ---
    plt.figure(figsize=(12, 6))
    plt.plot(np.arange(common_len), y_test_common, label='Original AQI',
             color='blue', linestyle='-')
    plt.plot(np.arange(common_len), fl_preds_common, label='FL Prediction',
             color='red', linestyle='--')
    plt.plot(np.arange(common_len), cent_preds_common, label='CL Prediction',
             color='green', linestyle='-.')
    plt.xlabel("Test Sample Index")
    plt.ylabel("AQI")
    plt.title(f"Prediction Comparison for City {city_name}")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
for city_idx in cities:
    # Choose a city (using the first city as an example)
    city = list(nodes_test_data.keys())[city_idx]
    city_name = city_mapping[city]
    X_test, y_test = nodes_test_data[city]

    # Get predictions from the personalized FL model
    fl_model = personalized_models[city]
    fl_preds = fl_model.predict(X_test, verbose=0).flatten()

    # Get predictions from the centralized model on the same test set
    # Here we assume that centralized model predictions can be obtained on X_test directly.
    # (If the centralized model uses a different windowing, you should ensure the comparison is on identical test samples.)
    cent_preds = model.predict(X_test, batch_size=32, verbose=0).flatten()

    # Compute the squared error for each sample for both models
    error_fl = (y_test - fl_preds)**2
    error_cl = (y_test - cent_preds)**2

    # Perform paired t-test to compare the errors
    t_stat, p_value = ttest_rel(error_fl, error_cl)

    print(f"City {city_name} - Paired t-test comparing FL and CL errors:")
    print(f"t-statistic: {t_stat:.4f}, p-value: {p_value:.4f}")